# Purpose

The purpose of this script is to analyze stock following principals outlines by Benjamin Graham. These principles involve the following guidlines

- PE ratio below 9 but positivie
- Current Ratio (Current Assets/Current Liab) greater than 1.5
- Debt Load (Total Debt/ Current Assets) Less than or equal to 1.1
- Positive EPS for each quarter in the last year
- Price To Book ratio less than or equal to 1.2 
- Look for dividends

Additionally I will be calculating the following variables for my own interest
- NCAV (Net Current Assets Value) = (Net Assets-TotalLiabilities-preferred stock)

## Load Name of Stocks
The section uses the yahoo finance library to get the names of many stocks. 

In [3]:
import yahoo_fin.stock_info as si
dow_list = si.tickers_dow() #Loads Tickers in dow jones
nasdaq_list = si.tickers_nasdaq() #Loads Tickers in NASDAQ
sp500_list = si.tickers_sp500()
other_list = si.tickers_other() #Loads Tickers in Other Indexes

## Load yfinance and Create Functions To Read Data I am interested In 
The yfinance library is what will primarily be used to load information for each of the stock tickers. Within this libary 3 primary functions will be called they are   

- stk.info #(used for PE, EPS, Price to Book values)
- stk.balance_sheet (used for Current Ratio, Debt Load, NCAV)
- stk.dividends (used for dividend calculations

In order to use these functions more efficiently they will be integrated into a number of my own creation. The first of these functions is below and is used to read useful information from the stk.info call

In [4]:
import yfinance as yf
import numpy
import pandas as pd

def extractInfo(Ticker):
    stk = 0
    commonStock = False
    divRate = False
    forPE = 0
    trailPE = 0
    forEPS = 0
    trailEPS = 0
    P2B = 0
    
    #Ticers with these values in their name were often for funds preferred stock etc
    #So this line removes those other investment mediums
    if '$' in Ticker or '.' in Ticker:
        return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk
    
    stk = yf.Ticker(Ticker) #Creates object to load stock data
    stkInfo = stk.info #Loads basic stock info 
    
    #In yfinance all common stock have forwardPE listed but funds and other investments do not.
    #So this line removes those other investment mediums
    if 'forwardPE' not in stkInfo or stkInfo['forwardPE'] == 'None': 
        return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk
    
    commonStock = True
    
    #Reads stock info where it is available to be read
    divRate = stkInfo['dividendRate'] if 'dividendRate' in stkInfo else 'None'
    forPE = stkInfo['forwardPE']
    trailPE = stkInfo['trailingPE'] if 'trailingPE' in stkInfo else 'Negative'
    forEPS = stkInfo['forwardEps'] if 'forwardEps' in stkInfo else 'None'
    trailEPS = stkInfo['trailingEps'] if 'trailingEps' in stkInfo else 'None'
    P2B = stkInfo['priceToBook'] if 'priceToBook' in stkInfo else 'None'
    
    return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk

The second function will be responsible for reading the balance sheet for the stk. It is assumed here that the function extractInfo has already been read ahead of this function and has prevented any non-common-stock investments from progressing further into the data extraction process 

In [5]:
def extractBalSheet(stk):
    stkBal= stk.balancesheet #Returns last four years of annual balance sheet
    if stkBal.empty:
        stkHoldEqty = 'None'
        curRat = 'None'
        debtLoad = 'None'
        curAsts = 'None'
        curLiabs = 'None'
        totAsts = 'None'
        totLiabs = 'None'  
        return stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs
    
    recBal = stkBal[stkBal.columns[0]] #Returns the most recent balance sheet
    
    curAsts = recBal['Total Current Assets'] if 'Total Current Assets' in recBal else 'None'
    curLiabs = recBal['Total Current Liabilities'] if 'Total Current Liabilities' in recBal else 'None'
    totAsts = recBal['Total Assets'] if 'Total Assets' in recBal else 'None'
    totLiabs = recBal['Total Liab'] if 'Total Liab' in recBal else 'None'
    
    stkHoldEqty = totAsts-totLiabs if (type(totAsts)!=str and type(totLiabs)!=str) else 'None'
    curRat = curAsts/curLiabs if (type(curAsts)!=str and type(curLiabs)!=str and curLiabs!=0) else 'None'
    debtLoad = totLiabs/curAsts if (type(curAsts)!=str and type(totLiabs)!=str and curAsts!=0) else 'None'
    
    return stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs
    

The third function is responsible for reading the earnings of the data for the last four monthes.

In [9]:
def extractPrev4QuartEarnings(stk):
    stkQuartEarn = stk.quarterly_earnings
    earn = stkQuartEarn['Earnings'] if 'Earnings' in stkQuartEarn.columns else None
    numQ = earn.shape[0] if earn is not None else 0
    mostRec = earn[-1] if numQ >= 1 else 'None' #Most recent is last value
    secMostRec = earn[-2] if numQ >= 2 else 'None'
    thrdMostRec = earn[-3] if numQ >= 3 else 'None'
    frthMostRec = earn[-4] if numQ >= 4 else 'None'
    
    return mostRec, secMostRec, thrdMostRec, frthMostRec
    

Get the Most Recent Quarters and Years using a popular stock which should have all the needed data

In [11]:
def extractQuartNamesMSFT():
    msft = yf.Ticker('MSFT')
    earn = msft.earnings
    qEarn = msft.quarterly_earnings
    yrs = 

SyntaxError: invalid syntax (<ipython-input-11-efdce4897f50>, line 5)

## Loop Through Stocks in the Dow

In [14]:
stkDFwFor = pd.DataFrame(columns = ['Ticker', 'divRate','forward PE','Trail PE','Forward EPS','Trail EPS','Price to Book',
                            'stock Holder Equity','Current Ration','Debt Load','Current Assets',
                            'Current Liabillities','Total Assets','Total Liabilities',
                            'Most Recent Quarter Earnings','2nd MRE','3rd MRE','4th MRE'])
stkDF = pd.DataFrame(columns = ['Ticker', 'divRate','Trail PE','Trail EPS','Price to Book',
                            'stock Holder Equity','Current Ratio','Debt Load','Current Assets',
                            'Current Liabillities','Total Assets','Total Liabilities',
                            'Most Recent Quarter Earnings','2nd MRE','3rd MRE','4th MRE'])

for Ticker in other_list:
    print(Ticker)
    commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk = extractInfo(Ticker)
    
    if commonStock == True:
        stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs = extractBalSheet(stk)
        mostRec, secMostRec, thrdMostRec, frthMostRec = extractPrev4QuartEarnings(stk)
        
        nrwFor = [Ticker,divRate,forPE,trailPE,forEPS,trailEPS,P2B,
             stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs,
             mostRec, secMostRec, thrdMostRec, frthMostRec]
        
        nr = [Ticker,divRate,trailPE,trailEPS,P2B,
              stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs,
              mostRec, secMostRec, thrdMostRec, frthMostRec]
        
        stkDFwFor = stkDFwFor.append(pd.Series(nrwFor,
                                      index = stkDFwFor.columns),
                            ignore_index=True)
        stkDF = stkDF.append(pd.Series(nr,
                                      index = stkDF.columns),
                            ignore_index=True)

A
AA
AAA
AAAU
AADR
AAIC
AAIC$B
AAIC$C
AAMC
AAN
AAP
AAT
AAU
AB
ABB
ABBV
ABC
ABEQ
ABEV
ABG
ABM
ABR
ABR$A
ABR$B
ABR$C
ABT
AC
ACA
ACB
ACC
ACCO
ACEL
ACES
ACH
ACI
ACIC.U
ACIO
ACM
ACN
ACND
ACND.U
ACND.W
ACP
ACRE
ACSG
ACSI
ACTV
ACU
ACV
ACVF
ACWF
ACWV
ACY
ADC
ADCT
ADFI
ADM
ADME
ADNT
ADS
ADT
ADX
AE
AEB
AEE
AEF
AEFC
AEG
AEL
AEL$A
AEL$B
AEM
AENZ
AEO
AER
AES
AESR
AFB
AFC
AFG
AFGB
AFGC
AFGD
AFGE
AFI
AFIF
AFK
AFL
AFLG
AFMC
AFSM
AFT
AFTY
AG
AGCO
AGD
AGE
AGG
AGGP
AGGY
AGI
AGM
AGM$C
AGM$D
AGM$E
AGM$F
AGM.A
AGO
AGO$B
AGO$E
AGO$F
AGQ
AGR
AGRO
AGS
AGT
AGX
AGZ
AHC
AHH
AHH$A
AHL$C
AHL$D
AHL$E
AHT
AHT$D
AHT$F
AHT$G
AHT$H
AHT$I
AI
AIC
AIEQ
AIF
AIG
AIG$A
AIG.W
AIIQ
AIM
AIN
AINC
AIO
AIR
AIRC.V
AIRI
AIT
AIV
AIV.V
AIW
AIZ
AIZN
AIZP
AJAX.U
AJG
AJRD
AJX
AJXA
AKO.A
AKO.B
AKR
AL
AL$A
ALB
ALC
ALE
ALEX
ALFA
ALG
ALIN$A
ALIN$B
ALIN$E
ALK
ALL
ALL$B
ALL$G
ALL$H
ALL$I
ALLE
ALLY
ALLY$A
ALP$Q
ALSN
ALTG
ALTG.W
ALTL
ALTS
ALUS
ALUS.U
ALUS.W
ALV
ALX
AM
AMBC
AMBC.W
AMBO
AMC
AMCR
AME
AMER
AMG
AMH
AMH$D
AMH$E
AMH$F
AMH$G
AMH$H
AMJ
AM

KeyboardInterrupt: 

## Save the Data Original Data Before Filtering

In [76]:
import os
name = 'FinalYahoo.ipynb'
dir_path = os.path.dirname(os.path.realpath(name)) #Finds directory of this file
if not os.path.isdir('./Excel/Raw'):
    os.mkdir('./Excel/Raw')
    
filename = 'other_12_10_2020.xls'
filepath = dir_path+'/Excel/Raw/'+filename
stkDF.to_excel(filepath)

## Filters Stocks Based on Benjamin Graham Requirements

Create a function to filter the stock

In [77]:
def filterStock(stkDF,maxPE,minCurRat,maxDebtLoad,maxPB):
    if type(data['Trail PE']) == str or type(data['Debt Load'])==str or type(data['Price to Book']) == str:
        return False
    if data['Trail PE'] is None or data['Debt Load'] is None or data['Price to Book'] is None:
        return False
    if data['Trail PE'] > maxPE:
        return False
    if data['Debt Load'] > maxDebtLoad:
        return False
    if data['Price to Book'] > maxPB:
        return False
    return True

In [78]:
maxPE = 15 #Graham Suggests 9
minCurRat = 1.5 #Graham Suggests 1.5
maxDebtLoad = 2 #Graham Suggests 1.1
maxPB = 2 #Graham Suggests 1.2

stkDF.index = stkDF['Ticker']
tkrList = stkDF.index

stkDfFltr = pd.DataFrame(columns = ['Ticker', 'divRate','Trail PE','Trail EPS','Price to Book',
                            'stock Holder Equity','Current Ratio','Debt Load','Current Assets',
                            'Current Liabillities','Total Assets','Total Liabilities',
                            'Most Recent Quarter Earnings','2nd MRE','3rd MRE','4th MRE'])

for tkr in tkrList:
    data = stkDF.loc[tkr]
    if filterStock(stkDF,maxPE,minCurRat,maxDebtLoad,maxPB):
        stkDfFltr = stkDfFltr.append(pd.Series(data,
                                      index = stkDfFltr.columns),
                            ignore_index=True)
    

## Save the Filtered Data Set

In [79]:
name = 'FinalYahoo.ipynb'
dir_path = os.path.dirname(os.path.realpath(name)) #Finds directory of this file
if not os.path.isdir('./Excel/Filtered'):
    os.mkdir('./Excel/Filtered')
    
filename = 'other_12_10_2020.xls'
filepath = dir_path+'/Excel/Filtered/'+filename
stkDfFltr.to_excel(filepath)

In [70]:
caci = yf.Ticker('CACI')
caci.info
extractInfo('CACI')

(True,
 None,
 15.012995,
 17.811468,
 16.16,
 13.621,
 2.1975145,
 yfinance.Ticker object <CACI>)

In [ ]:
msft.cashflow